In [114]:
import numpy as np
import pandas as pd
import sqlite3
import sys
import os
import re
from urllib.parse import urlparse

In [181]:
data_path = 'exp-data/nyt-c1.sqlite'

In [182]:
query = """SELECT sv.site_url, sv.visit_id,
        hr.headers, hr.referrer, hr.url, hr.top_level_url
        FROM http_requests as hr LEFT JOIN site_visits as sv
        ON sv.visit_id = hr.visit_id
        """

In [183]:
conn = sqlite3.connect(data_path)
# http_requests = pd.read_sql_query("select visit_id, headers, referrer, url, top_level_url from http_requests;", conn)
http_requests = pd.read_sql_query(query, conn)

In [184]:
http_requests

,site_url,visit_id,headers,referrer,url,top_level_url
0,https://www.webmd.com/,1,"[[""Host"",""www.webmd.com""],[""User-Agent"",""Mozil...",,https://www.webmd.com/,https://www.webmd.com/
1,https://www.webmd.com/,1,"[[""Host"",""img.webmd.com""],[""User-Agent"",""Mozil...",https://www.webmd.com/,https://img.webmd.com/dtmcms/live/webmd/consum...,https://www.webmd.com/
2,https://www.webmd.com/,1,"[[""Host"",""css.webmd.com""],[""User-Agent"",""Mozil...",https://www.webmd.com/,https://css.webmd.com/dtmcms/live/webmd/PageBu...,https://www.webmd.com/
3,https://www.webmd.com/,1,"[[""Host"",""css.webmd.com""],[""User-Agent"",""Mozil...",https://www.webmd.com/,https://css.webmd.com/dtmcms/live/webmd/PageBu...,https://www.webmd.com/
4,https://www.webmd.com/,1,"[[""Host"",""css.webmd.com""],[""User-Agent"",""Mozil...",https://www.webmd.com/,https://css.webmd.com/dtmcms/live/webmd/PageBu...,https://www.webmd.com/
5,https://www.webmd.com/,1,"[[""Host"",""js.webmd.com""],[""User-Agent"",""Mozill...",https://www.webmd.com/,https://js.webmd.com/dtmcms/live/webmd/PageBui...,https://www.webmd.com/
6,https://www.webmd.com/,1,"[[""Host"",""js.webmd.com""],[""User-Agent"",""Mozill...",https://www.webmd.com/,https://js.webmd.com/dtmcms/live/webmd/PageBui...,https://www.webmd.com/
7,https://www.webmd.com/,1,"[[""Host"",""assets.adobedtm.com""],[""User-Agent"",...",https://www.webmd.com/,https://assets.adobedtm.com/2c8c1e17b98c/e6d47...,https://www.webmd.com/
8,https://www.webmd.com/,1,"[[""Host"",""img.webmd.com""],[""User-Agent"",""Mozil...",https://www.webmd.com/,https://img.webmd.com/dtmcms/live/webmd/consum...,https://www.webmd.com/
9,https://www.webmd.com/,1,"[[""Host"",""img.webmd.com""],[""User-Agent"",""Mozil...",https://www.webmd.com/,https://img.webmd.com/dtmcms/live/webmd/consum...,https://www.webmd.com/


In [185]:
cookies = pd.read_sql_query("select visit_id, record_type, change_cause, host, name, value from javascript_cookies;", conn)

In [186]:
def get_host_from_headers(x):
    temp = x.replace('"','').split('],[')
    for t in temp:
            pair = t.replace(']]','').replace('[[','').split(',')
            if "Host" in pair:
                host_value = pair[1]
    return host_value

def get_cookies_from_headers(x):
    ls = x.replace('"','').split('],[')
    for l in ls:
        ta = l.replace(']]','').split(',')
        if "Cookie" in ta:
            return ta[-1]
    return ''
def extract_host_from_url(url_ls):
    return list(map(lambda x: urlparse(x).netloc.split('.')[1] if len(urlparse(x).netloc.split('.')) > 1 else x, url_ls))

def get_site(site_url):
    return site_url.split(".")[1]
def is_third_party(origin, url_host):
    if origin in url_host:
        return False
    return True

In [187]:
http_requests["host"] = list(map(lambda x: get_host_from_headers(x), http_requests.headers.tolist()))
http_requests["cookies"] =  list(map(lambda x: get_cookies_from_headers(x), http_requests.headers.tolist()))
http_requests["origin_site"] = list(map(lambda x: get_site(x), http_requests.site_url))
http_requests["is_third_party"] = list(map(lambda a, b: is_third_party(a, b), http_requests.origin_site, http_requests.host))

In [188]:
http_requests_third_party = http_requests[http_requests.is_third_party == True]

In [189]:
http_requests_third_party

,site_url,visit_id,headers,referrer,url,top_level_url,host,cookies,origin_site,is_third_party
7,https://www.webmd.com/,1,"[[""Host"",""assets.adobedtm.com""],[""User-Agent"",...",https://www.webmd.com/,https://assets.adobedtm.com/2c8c1e17b98c/e6d47...,https://www.webmd.com/,assets.adobedtm.com,,webmd,True
51,https://www.webmd.com/,1,"[[""Host"",""www.googleadservices.com""],[""User-Ag...",https://www.webmd.com/,https://www.googleadservices.com/pagead/conver...,https://www.webmd.com/,www.googleadservices.com,,webmd,True
53,https://www.webmd.com/,1,"[[""Host"",""fonts.googleapis.com""],[""User-Agent""...",https://css.webmd.com/dtmcms/live/webmd/PageBu...,https://fonts.googleapis.com/css?family=Roboto...,https://www.webmd.com/,fonts.googleapis.com,,webmd,True
54,https://www.webmd.com/,1,"[[""Host"",""fonts.googleapis.com""],[""User-Agent""...",https://css.webmd.com/dtmcms/live/webmd/PageBu...,https://fonts.googleapis.com/css?family=Roboto...,https://www.webmd.com/,fonts.googleapis.com,,webmd,True
55,https://www.webmd.com/,1,"[[""Host"",""dyv1bugovvq1g.cloudfront.net""],[""Use...",https://www.webmd.com/,https://dyv1bugovvq1g.cloudfront.net/25/www.we...,https://www.webmd.com/,dyv1bugovvq1g.cloudfront.net,,webmd,True
56,https://www.webmd.com/,1,"[[""Host"",""d15kdpgjg3unno.cloudfront.net""],[""Us...",https://www.webmd.com/,https://d15kdpgjg3unno.cloudfront.net/oPS.js?c...,https://www.webmd.com/,d15kdpgjg3unno.cloudfront.net,,webmd,True
57,https://www.webmd.com/,1,"[[""Host"",""fonts.gstatic.com""],[""User-Agent"",""M...",https://fonts.googleapis.com/css?family=Roboto...,https://fonts.gstatic.com/s/sourcesanspro/v14/...,https://www.webmd.com/,fonts.gstatic.com,,webmd,True
58,https://www.webmd.com/,1,"[[""Host"",""securepubads.g.doubleclick.net""],[""U...",https://www.webmd.com/,https://securepubads.g.doubleclick.net/tag/js/...,https://www.webmd.com/,securepubads.g.doubleclick.net,,webmd,True
59,https://www.webmd.com/,1,"[[""Host"",""contextual.media.net""],[""User-Agent""...",https://www.webmd.com/,https://contextual.media.net/bidexchange.js?ci...,https://www.webmd.com/,contextual.media.net,,webmd,True
60,https://www.webmd.com/,1,"[[""Host"",""z.moatads.com""],[""User-Agent"",""Mozil...",https://www.webmd.com/,https://z.moatads.com/webmdheader894912230343/...,https://www.webmd.com/,z.moatads.com,,webmd,True


In [190]:
query2 = """SELECT sv.site_url, sv.visit_id,
        jsc.host, jsc.name, jsc.value
        FROM javascript_cookies as jsc LEFT JOIN site_visits as sv
        ON sv.visit_id = jsc.visit_id
        """
cookies = pd.read_sql_query(query2, conn)

In [191]:
# query2 = "select visit_id, record_type, change_cause, host, name, value, time_stamp from javascript_cookies;"
# 
cookies['origin_site'] = list(map(lambda x: get_site(x), cookies.site_url))
cookies['is_third_party'] = list(map(lambda a, b: is_third_party(a, b), cookies.origin_site, cookies.host))

In [192]:
cookies_first_party = cookies[cookies.is_third_party == False]

In [193]:
cookies_first_party

,site_url,visit_id,host,name,value,origin_site,is_third_party
0,https://www.webmd.com/,1,.www.webmd.com,__cfduid,dd673b2700baa134208e3794c02eec1f41606506583,webmd,False
1,https://www.webmd.com/,1,.webmd.com,lrt_wrk,lrt1_cached_k8_worker_1_37S_21O_2020-11-18_17:...,webmd,False
2,https://www.webmd.com/,1,.webmd.com,gtinfo,"{""ct"":""Columbus"",""c"":""Franklin"",""cc"":""39049"",""...",webmd,False
3,https://www.webmd.com/,1,.webmd.com,VisitorId,9831e05f-086b-4123-5987-f9b5df875967,webmd,False
4,https://www.webmd.com/,1,.css.webmd.com,__cfduid,dc560e081e9789ea111079dc63ad819771606506583,webmd,False
5,https://www.webmd.com/,1,.img.webmd.com,__cfduid,dcbcfc026b947368f44e61473dc7f86bd1606506583,webmd,False
6,https://www.webmd.com/,1,.js.webmd.com,__cfduid,dd75813509c3a8449208fe627853f63d41606506583,webmd,False
7,https://www.webmd.com/,1,.webmd.com,test,cookie,webmd,False
8,https://www.webmd.com/,1,.webmd.com,test,cookie,webmd,False
9,https://www.webmd.com/,1,.webmd.com,test,cookie,webmd,False


In [194]:
# cookie_effective_values = [v.replace('|', '\|').replace("-", "\-").replace("[",'\[').replace(']','\]') for v in set(cookies.value) if len(v) > 5]
cookie_effective_values = [re.escape(v) for v in set(cookies_first_party.value) if len(v) > 5]
cookie_effective_all = [re.escape(v) for v in set(cookies.value) if len(v) > 5]

In [195]:
cookie_values = "|".join(cookie_effective_values)
#cookie_values = "|".join(cookie_effective_all)

In [196]:
http_requests_syncs = http_requests_third_party[http_requests_third_party.url.str.contains(cookie_values, regex=True)]

In [197]:
# extract third party
http_requests_syncs['referrer_host'] = list(map(lambda x: urlparse(x).netloc, http_requests_syncs.referrer))

/Users/rainyleon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [198]:
# can tell wether referrer and url are different
http_requests_syncs['is_diff_host'] = list(map(lambda a, b: a != b, http_requests_syncs.host, http_requests_syncs.referrer_host))

/Users/rainyleon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [199]:
http_requests_syncs['referred_by_third'] = list(map(lambda a, b: is_third_party(a, b), http_requests_syncs.origin_site, http_requests_syncs.referrer_host))

/Users/rainyleon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [200]:
http_requests_syncs[http_requests_syncs.is_diff_host][http_requests_syncs.visit_id==3]

/Users/rainyleon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,site_url,visit_id,headers,referrer,url,top_level_url,host,cookies,origin_site,is_third_party,referrer_host,is_diff_host,referred_by_third
2459,https://www.nytimes.com/,3,"[[""Host"",""www.googletagmanager.com""],[""User-Ag...",https://www.nytimes.com/,https://www.googletagmanager.com/gtm.js?id=GTM...,https://www.nytimes.com/,www.googletagmanager.com,,nytimes,True,www.nytimes.com,True,False
2547,https://www.nytimes.com/,3,"[[""Host"",""5290727.fls.doubleclick.net""],[""User...",https://www.nytimes.com/,https://5290727.fls.doubleclick.net/activityi;...,https://www.nytimes.com/,5290727.fls.doubleclick.net,test_cookie=CheckForPermission,nytimes,True,www.nytimes.com,True,False
2569,https://www.nytimes.com/,3,"[[""Host"",""www.google-analytics.com""],[""User-Ag...",https://www.nytimes.com/,https://www.google-analytics.com/j/collect?v=1...,https://www.nytimes.com/,www.google-analytics.com,,nytimes,True,www.nytimes.com,True,False
2570,https://www.nytimes.com/,3,"[[""Host"",""stags.bluekai.com""],[""User-Agent"",""M...",https://www.nytimes.com/,https://stags.bluekai.com/site/50136?limit=1&i...,https://www.nytimes.com/,stags.bluekai.com,bkdc=phx; bku=5LD99cNjyPI0ZERF,nytimes,True,www.nytimes.com,True,False
2573,https://www.nytimes.com/,3,"[[""Host"",""adservice.google.com""],[""User-Agent""...",https://5290727.fls.doubleclick.net/activityi;...,https://adservice.google.com/ddm/fls/z/src=529...,https://www.nytimes.com/,adservice.google.com,NID=204=yMh99bQ9AG11gArh5VZA32ANpUgvui-T9uZOxW...,nytimes,True,5290727.fls.doubleclick.net,True,True
2606,https://www.nytimes.com/,3,"[[""Host"",""securepubads.g.doubleclick.net""],[""U...",https://www.nytimes.com/,https://securepubads.g.doubleclick.net/gampad/...,https://www.nytimes.com/,securepubads.g.doubleclick.net,IDE=AHWqTUli2Ug4RelkdVQsl6mdMiY18CRQy0CVBjgL0Z...,nytimes,True,www.nytimes.com,True,False
2607,https://www.nytimes.com/,3,"[[""Host"",""securepubads.g.doubleclick.net""],[""U...",https://www.nytimes.com/,https://securepubads.g.doubleclick.net/gampad/...,https://www.nytimes.com/,securepubads.g.doubleclick.net,IDE=AHWqTUli2Ug4RelkdVQsl6mdMiY18CRQy0CVBjgL0Z...,nytimes,True,www.nytimes.com,True,False
2823,https://www.nytimes.com/,3,"[[""Host"",""cookiex.ngd.yahoo.com""],[""User-Agent...",https://eus.rubiconproject.com/usync.html?p=me...,https://cookiex.ngd.yahoo.com/ack?xid=p.Jms4Yn...,https://www.nytimes.com/,cookiex.ngd.yahoo.com,A3=d=AQABBIdZwV8CEAePsjTU5cRNE7tmBgFkr9IFEgEBA...,nytimes,True,eus.rubiconproject.com,True,True


In [180]:
http_requests_syncs[http_requests_syncs.is_diff_host][http_requests_syncs.visit_id==3]

/Users/rainyleon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,site_url,visit_id,headers,referrer,url,top_level_url,host,cookies,origin_site,is_third_party,referrer_host,is_diff_host,referred_by_third
2824,https://www.nytimes.com/,3,"[[""Host"",""www.googletagmanager.com""],[""User-Ag...",https://www.nytimes.com/,https://www.googletagmanager.com/gtm.js?id=GTM...,https://www.nytimes.com/,www.googletagmanager.com,,nytimes,True,www.nytimes.com,True,False
2904,https://www.nytimes.com/,3,"[[""Host"",""5290727.fls.doubleclick.net""],[""User...",https://www.nytimes.com/,https://5290727.fls.doubleclick.net/activityi;...,https://www.nytimes.com/,5290727.fls.doubleclick.net,,nytimes,True,www.nytimes.com,True,False
2922,https://www.nytimes.com/,3,"[[""Host"",""adservice.google.com""],[""User-Agent""...",https://5290727.fls.doubleclick.net/activityi;...,https://adservice.google.com/ddm/fls/i/src=529...,https://www.nytimes.com/,adservice.google.com,NID=204=D54h98AZvmH89boY4AOEYkgO1Yu0fqMzWQqU7n...,nytimes,True,5290727.fls.doubleclick.net,True,True
2929,https://www.nytimes.com/,3,"[[""Host"",""stags.bluekai.com""],[""User-Agent"",""M...",https://www.nytimes.com/,https://stags.bluekai.com/site/50136?limit=1&i...,https://www.nytimes.com/,stags.bluekai.com,bkdc=phx; bku=5LD99nOUyPI+cnRG,nytimes,True,www.nytimes.com,True,False
2936,https://www.nytimes.com/,3,"[[""Host"",""www.google-analytics.com""],[""User-Ag...",https://www.nytimes.com/,https://www.google-analytics.com/j/collect?v=1...,https://www.nytimes.com/,www.google-analytics.com,,nytimes,True,www.nytimes.com,True,False
2975,https://www.nytimes.com/,3,"[[""Host"",""securepubads.g.doubleclick.net""],[""U...",https://www.nytimes.com/,https://securepubads.g.doubleclick.net/gampad/...,https://www.nytimes.com/,securepubads.g.doubleclick.net,IDE=AHWqTUmi4RVj356DXmEaC4Hzv7dq2CVyPkwHafkI0q...,nytimes,True,www.nytimes.com,True,False
2976,https://www.nytimes.com/,3,"[[""Host"",""securepubads.g.doubleclick.net""],[""U...",https://www.nytimes.com/,https://securepubads.g.doubleclick.net/gampad/...,https://www.nytimes.com/,securepubads.g.doubleclick.net,IDE=AHWqTUmi4RVj356DXmEaC4Hzv7dq2CVyPkwHafkI0q...,nytimes,True,www.nytimes.com,True,False
3182,https://www.nytimes.com/,3,"[[""Host"",""cookiex.ngd.yahoo.com""],[""User-Agent...",https://eus.rubiconproject.com/usync.html?p=me...,https://cookiex.ngd.yahoo.com/ack?xid=4.FSGr1x...,https://www.nytimes.com/,cookiex.ngd.yahoo.com,A3=d=AQABBLVXwV8CEO2CuGwdUrBvVWgkVOVIuUIFEgEBA...,nytimes,True,eus.rubiconproject.com,True,True
3228,https://www.nytimes.com/,3,"[[""Host"",""p.adsymptotic.com""],[""User-Agent"",""M...",https://ads.pubmatic.com/AdServer/js/showad.js,https://p.adsymptotic.com/d/px?_pid=10291&_psi...,https://www.nytimes.com/,p.adsymptotic.com,U=8404820d8b2b85b7e285c2b1d4449fa0,nytimes,True,ads.pubmatic.com,True,True
3240,https://www.nytimes.com/,3,"[[""Host"",""pagead2.googlesyndication.com""],[""Us...",https://eb6611b34a41a87fee3f06cd0e100508.safef...,https://pagead2.googlesyndication.com/pagead/s...,https://www.nytimes.com/,pagead2.googlesyndication.com,,nytimes,True,eb6611b34a41a87fee3f06cd0e100508.safeframe.goo...,True,True
